<a href="https://colab.research.google.com/github/kushagra-atre/Metal-Price-API/blob/main/API_Metal_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Replace with your actual API key
api_key = 'd45ac101b1a5e9fea482e31cf68538e0'

# Initialize the start and end date for the loop
start_date = datetime(2013, 1, 1)
end_date = datetime(2023, 11, 3)

# Initialize a list to store all the DataFrame chunks
data_chunks = []

while start_date < end_date:
    # Calculate the next year's date or the end date, whichever is sooner
    next_year_date = min(start_date + timedelta(days=365), end_date)

    # Format dates as strings in the required format
    start_date_str = start_date.strftime('%Y-%m-%d')
    next_year_date_str = next_year_date.strftime('%Y-%m-%d')

    # Construct the API URL
    url = f'https://api.metalpriceapi.com/v1/timeframe?api_key={api_key}&start_date={start_date_str}&end_date={next_year_date_str}&base=USD&currencies=XAG'

    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Check if the API request was successful
        if data.get('success'):
            # Convert the rates to a pandas DataFrame
            df = pd.DataFrame(data['rates']).T
            # Add the DataFrame to the list of chunks
            data_chunks.append(df)
        else:
            print(f"API request failed for {start_date_str} to {next_year_date_str}:", data.get('error'))
    else:
        print(f"Failed to fetch data for {start_date_str} to {next_year_date_str}:", response.status_code)

    # Update the start_date to the next year's date for the next iteration
    start_date = next_year_date + timedelta(days=1)

# Concatenate all the DataFrame chunks
all_data = pd.concat(data_chunks)

# Reset the index of the all_data DataFrame and rename the columns
all_data.reset_index(inplace=True)
all_data.rename(columns={'index': 'Date', 'XAG': 'Silver Price'}, inplace=True)

# Save the DataFrame to a CSV file
all_data.to_csv('silver_prices_2013_2023.csv', index=False)

print("Data fetching complete. The data is saved to 'silver_prices_2013_2023.csv'.")

# Now you can proceed with your analysis and forecasting with the DataFrame `all_data`


Data fetching complete. The data is saved to 'silver_prices_2013_2023.csv'.
